# Beamforming animation

In [52]:
import numpy as np
import plotly.graph_objects as go
import beamforming

## Create base signal tx_signal with f = 20kHz

In [53]:
sample_rate = 1e6
N = 10000 # number of samples to simulate

# Create a tone to act as the transmitter signal
t = np.arange(N)/sample_rate # time vector
f_tone = 0.02e6
tx_signal = beamforming.create_IQ_signal(f_tone, t)

## Generate all beams

In [54]:
results = list()

angles = np.arange(-90, 100, 10)
nb_antenas = 10
d_antenas = 0.5

for theta in angles:
    results.append(beamforming.calculate_beam(tx_signal, nb_antenas, d_antenas, theta))

theta_scan_deg = np.degrees(np.linspace(-1*np.pi, np.pi, 1000))

In [55]:
scan = list()
for i, theta in enumerate(angles):
    scan.append(list([None]*len(results[i])))
    index = np.absolute(theta_scan_deg-theta).argmin()
    scan[-1][index-1] = results[i].min()
    scan[-1][index] = 0

## Plot results

In [56]:
fig = go.Figure()
fig.add_trace(go.Scatterpolar(r=results[0], theta=theta_scan_deg, name='scan', mode='lines', showlegend=True))

fig.add_trace(go.Scatterpolar(r=scan[0], theta=theta_scan_deg, name='beam', mode='lines', showlegend=True, 
                                  textposition="top right", textfont=dict(color="Red") ))

fig.update_layout(polar = dict(radialaxis_angle = 90, angularaxis = dict(direction = "clockwise" ), sector=[-180, 180]))
fig.update_layout(template="plotly_dark")

fig.update_layout(updatemenus=[dict(
                  type="buttons",
                  buttons=[dict(label="Play",
                              method="animate",
                              args=[None])])])

base_text = str(nb_antenas) + " antenas " + "d=" + str(d_antenas) + " "
frames = list()
for i, r in enumerate(results):
    frames.append( go.Frame(data=[go.Scatterpolar(r=r), go.Scatterpolar(r=scan[i])], layout=go.Layout(title_text=base_text+str(angles[i])+"°") ) )
fig.update(frames=frames)
# fig.show(renderer='browser')  
fig.write_html(base_text + ".html")